In [6]:
import torch
import torch.nn as nn
import torch.onnx

# Define the PyTorch model
class ConvSoftmaxModel(nn.Module):
    def __init__(self):
        super(ConvSoftmaxModel, self).__init__()
        # Define the convolution layer
        self.conv = nn.Conv2d(in_channels=16, out_channels=2, kernel_size=3, stride=1, padding=0)
        # Define the softmax layer
        self.softmax = nn.Softmax(dim=1)  # Softmax along the channel dimension

    def forward(self, x):
        x = self.conv(x)  # Apply convolution
        # x = self.softmax(x)  # Apply softmax
        return x

# Initialize the model
model = ConvSoftmaxModel()

# Set model to evaluation mode
model.eval()

# Create a dummy input tensor with the specified size
dummy_input = torch.randn(1, 16, 28, 28)

# Export to ONNX
onnx_file_name = "conv_model.onnx"
torch.onnx.export(
    model,                     # PyTorch model
    dummy_input,               # Input tensor
    onnx_file_name,            # Output ONNX file name
    export_params=True,        # Store the trained weights in the model file
    opset_version=17,          # Specify the ONNX opset version
    do_constant_folding=True,  # Optimize the model by folding constants
    input_names=['input'],     # Input tensor name
    output_names=['output'],   # Output tensor name
    dynamic_axes={             # Allow for dynamic axes (optional)
        'input': {0: 'batch_size'},  # Batch size can be dynamic
        'output': {0: 'batch_size'}
    }
)

print(f"Model has been exported to {onnx_file_name}")

Model has been exported to conv_model.onnx


In [9]:
import onnx
import onnxruntime as ort
import numpy as np

# Load the ONNX model
onnx_model = onnx.load(onnx_file_name)
onnx.checker.check_model(onnx_model)  # Check the model for validity

# Run inference with ONNX Runtime
ort_session = ort.InferenceSession(onnx_file_name)

# Prepare input for ONNX Runtime
onnx_input = {"input": dummy_input.numpy()}
onnx_output = ort_session.run(None, onnx_input)

# Save the input data as an .npz file with the key "input"
np.savez("random_input.npz", input=dummy_input.numpy())

print("ONNX model output:", onnx_output)

ONNX model output: [array([[[[ 0.5898721 , -0.4970696 ,  1.7205774 , ..., -0.13172416,
          -0.42879182, -0.24764721],
         [ 0.15201464,  0.13163836, -1.2783546 , ...,  0.19562598,
          -0.27487624, -0.9580162 ],
         [-0.10930234, -0.68348265,  0.9518752 , ...,  0.34615433,
           0.34760594,  0.4766491 ],
         ...,
         [ 0.60248625, -0.28725997, -0.2807573 , ..., -0.69982004,
           0.14739057,  0.71417487],
         [-0.22566801, -0.35248914,  0.03485831, ...,  0.70564187,
           1.1572269 , -0.76566   ],
         [-0.95269954,  0.0625494 ,  0.8252905 , ..., -0.24186622,
          -0.6250172 ,  0.48324895]],

        [[ 0.494749  , -0.38136667,  0.41475433, ..., -0.66632915,
           0.35288537,  0.00568765],
         [-0.25870886,  0.4251236 , -0.12258695, ..., -0.470303  ,
          -0.8744751 , -0.57713145],
         [-0.76608646, -0.54289764,  0.33878696, ..., -0.14981632,
          -0.8031319 , -0.6581247 ],
         ...,
         [ 0.0

In [5]:
import torch
import torch.nn as nn
import torch.onnx

# Define a custom PyTorch model for a single matrix multiplication
class MatrixMultiplicationModel(nn.Module):
    def __init__(self):
        super(MatrixMultiplicationModel, self).__init__()

    def forward(self, x, y):
        return torch.matmul(x, y)

# Initialize the model
model = MatrixMultiplicationModel()

# Create dummy inputs (1024, 1024) matrices
input_1 = torch.randn(1, 1, 1024, 1024)
input_2 = torch.randn(1, 1, 1024, 1024)

# Export the model to ONNX
onnx_file_path = "matmul.onnx"
torch.onnx.export(
    model,                           # The model
    (input_1, input_2),              # The model inputs
    onnx_file_path,                  # Path to save the ONNX file
    opset_version=17,                # Specify the ONNX opset version
    input_names=["input_1", "input_2"],  # Input tensor names
    output_names=["output"],         # Output tensor name
)

print(f"ONNX model has been saved to {onnx_file_path}")

ONNX model has been saved to matmul.onnx


In [6]:
import numpy as np

# Generate random inputs for the model
input_1 = np.random.randn(1, 1, 1024, 1024).astype(np.float32)  # Random float32 array (1024x1024)
input_2 = np.random.randn(1, 1, 1024, 1024).astype(np.float32)  # Random float32 array (1024x1024)

# Save the inputs as a .npz file
npz_file_path = "random_input.npz"
np.savez(npz_file_path, input_1=input_1, input_2=input_2)

print(f"Random inputs have been saved to {npz_file_path}")

Random inputs have been saved to random_input.npz


In [19]:
# Dimensions of the matrices
A = 1024
B = 1024
C = 1024
runtime_ms = 834

# FLOPS formula: 2 * A * B * C
flops = 2 * A * B * C
gflops = flops/10**9
gflop_pers = gflops / (runtime_ms*10**(-3))
print(f"model has {gflops:,} GOP")
print(f"model GOPS/s={gflop_pers:,}")

model has 2.147483648 GOP
model GOPS/s=2.5749204412470026
